### GA RMSE

In [258]:
import numpy as np
from sklearn.metrics import mean_squared_error

def roulette_wheel_selection(population, fitness_scores):
    """
    룰렛 휠 선택 방식으로 부모 개체를 선택
    Args:
        population: 현재 개체군 (가중치 배열)
        fitness_scores: 각 개체의 RMSE (낮을수록 좋음)
    Returns:
        선택된 부모 개체
    """
    fitness_list = []
    max_fitness = max(fitness_scores)
    min_fitness = min(fitness_scores)
    adjustment = (max_fitness - min_fitness) / 2 if max_fitness != min_fitness else 1  # 조정 값

    for m in range(len(fitness_scores)):
        # fitness 계산
        fitness = max_fitness - fitness_scores[m] + adjustment
        fitness_list.append(fitness)

    fitness_list = np.array(fitness_list)
    total_fitness = np.sum(fitness_list)

    # total_fitness가 0인 경우 예외 처리
    if total_fitness == 0:
        raise ValueError("Total fitness is 0, cannot calculate probabilities.")

    probabilities = fitness_list / total_fitness

    # 룰렛 휠 방식으로 부모 선택
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]


def genetic_algorithm_with_elitism(model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03):
    """
    GA를 사용하여 최적의 가중치 찾기
    Args:
        model_predictions: (n_samples, models, time_window, 2) 형태의 모델 예측값
        true_values: (n_samples, time_window, 2) 형태의 실제값
        population_size: 초기 개체군 크기
        generations: 세대 수
        mutation_rate: 돌연변이 확률
    Returns:
        최적 가중치 배열
    """
    num_models = model_predictions.shape[1]  # 모델 수
    population = np.random.dirichlet(np.ones(num_models), size=population_size)  # 초기 가중치 개체군

    for generation in range(generations):
        fitness_scores = []

        # 현재 population의 fitness (RMSE) 계산
        for individual in population:
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            rmse = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))
            fitness_scores.append(rmse)
        
        fitness_scores = np.array(fitness_scores)

        # 새로운 population 생성
        new_population = []
        while len(new_population) < population_size:
            # 부모 선택
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)

            # 교차 연산
            crossover_point = np.random.randint(1, num_models)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

            # 돌연변이
            if np.random.rand() < mutation_rate:
                mutation_vector = np.random.normal(0, 0.1, size=num_models)
                child = np.clip(child + mutation_vector, 0, 1)

            # 정규화
            child = child / np.sum(child)
            new_population.append(child)
        
        new_population = np.array(new_population)

        # 엘리티즘 적용: 상위 50% 개체 선택
        combined_population = np.vstack((population, new_population))
        combined_fitness = np.concatenate((fitness_scores, np.zeros(len(new_population))))

        for idx, individual in enumerate(new_population):
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            combined_fitness[len(fitness_scores) + idx] = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))

        sorted_indices = np.argsort(combined_fitness)
        population = combined_population[sorted_indices][:population_size]

        print(f"Generation {generation + 1}/{generations}, Best RMSE: {combined_fitness[sorted_indices][0]}")

    # 최적 개체 반환
    best_weights = population[0]
    print(f"Optimal Weights: {best_weights}")
    return best_weights


In [259]:
import os
import numpy as np
import pandas as pd

def load_and_process_validation_data(input_directory, month, models=5, time_window=48):
    """
    Validation 데이터를 로드하고, 모델 예측값과 True 값을 추출해 시간 단위로 이어붙입니다.

    Args:
        input_directory (str): 모델별 validation 데이터가 있는 디렉토리
        models (int): 모델 개수 (default=5)
        time_window (int): 샘플당 시간 창 (default=48)

    Returns:
        model_predictions (np.ndarray): (n_samples, models, time_window, 2)
        true_values (np.ndarray): (n_samples, time_window, 2)
    """
    model_predictions = []  # 각 모델의 예측값 저장

    for model_idx in range(1, models + 1):
        model_dir = os.path.join(input_directory, f"model{model_idx}")

        all_pred_u = []
        all_pred_v = []
        all_true_u = []
        all_true_v = []
        
        file_path = os.path.join(model_dir, f'val_month_{month}_model_{model_idx}_results.csv')
        df = pd.read_csv(file_path)

        # 예측값 가져오기
        pred_u = df[f"Model {model_idx} Val Pred U"].values
        pred_v = df[f"Model {model_idx} Val Pred V"].values

        all_pred_u.extend(pred_u)
        all_pred_v.extend(pred_v)

        # True 값은 모델 1에서만 추출 (중복 방지)
        if model_idx == 1:
            all_true_u.extend(df["True U"].values)
            all_true_v.extend(df["True V"].values)

        # 48시간 단위로 나누기
        n_samples = len(all_pred_u) // time_window
        pred_u = np.array(all_pred_u[:n_samples * time_window]).reshape(n_samples, time_window, 1)
        pred_v = np.array(all_pred_v[:n_samples * time_window]).reshape(n_samples, time_window, 1)

        model_predictions.append(np.concatenate([pred_u, pred_v], axis=2))  # (n_samples, 48, 2)

        # True 값 병합 (한 번만 실행)
        if model_idx == 1:
            true_u = np.array(all_true_u[:n_samples * time_window]).reshape(n_samples, time_window, 1)
            true_v = np.array(all_true_v[:n_samples * time_window]).reshape(n_samples, time_window, 1)
            true_values = np.concatenate([true_u, true_v], axis=2)  # (n_samples, 48, 2)

    # 모델 예측값 형태 변환: (n_samples, models, 48, 2)
    model_predictions = np.stack(model_predictions, axis=1)

    return model_predictions, true_values

In [260]:
# 새로운 블록: GA 실행 및 최적 가중치 찾기
input_directory = 'ga_w2'
optimal_weights_list=[]
for month in range(1, 12):
    try:

        model_predictions, true_values = load_and_process_validation_data(input_directory, month)

        if len(model_predictions) == 0 or len(true_values) == 0:
            print(f"{month}월: 데이터가 부족하여 건너뜁니다.")
            continue

        # GA 실행하여 최적 가중치 찾기
        optimal_weights = genetic_algorithm_with_elitism(
            model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03
        )
        optimal_weights_list.append(optimal_weights)
    
    except Exception as e:
        print(f"{month}월 처리 중 오류 발생: {e}")

Generation 1/100, Best RMSE: 4.7587338726650055
Generation 2/100, Best RMSE: 4.752860649967825
Generation 3/100, Best RMSE: 4.747961143065967
Generation 4/100, Best RMSE: 4.747386390200559
Generation 5/100, Best RMSE: 4.743072293855496
Generation 6/100, Best RMSE: 4.737992355048764
Generation 7/100, Best RMSE: 4.733915939069685
Generation 8/100, Best RMSE: 4.732685040820898
Generation 9/100, Best RMSE: 4.732068768906305
Generation 10/100, Best RMSE: 4.731124833193942
Generation 11/100, Best RMSE: 4.730397387501792
Generation 12/100, Best RMSE: 4.729806369318748
Generation 13/100, Best RMSE: 4.729766471485419
Generation 14/100, Best RMSE: 4.729766471485419
Generation 15/100, Best RMSE: 4.729712560219636
Generation 16/100, Best RMSE: 4.729656211198706
Generation 17/100, Best RMSE: 4.729652619964653
Generation 18/100, Best RMSE: 4.72964701131702
Generation 19/100, Best RMSE: 4.729632999611712
Generation 20/100, Best RMSE: 4.72963207168552
Generation 21/100, Best RMSE: 4.72963207168552
Gen

In [261]:
print(optimal_weights_list)

[array([0.10123664, 0.        , 0.74892202, 0.14984134, 0.        ]), array([0.32101635, 0.        , 0.        , 0.39395431, 0.28502935]), array([0.        , 0.        , 0.39715384, 0.60284616, 0.        ]), array([0.11855262, 0.88144738, 0.        , 0.        , 0.        ]), array([0.32416173, 0.57812405, 0.        , 0.09771422, 0.        ]), array([0.27629276, 0.60595773, 0.11774951, 0.        , 0.        ]), array([0.20503387, 0.3296041 , 0.        , 0.        , 0.46536203]), array([0.5640595 , 0.34967998, 0.08626052, 0.        , 0.        ]), array([0.24724964, 0.75275036, 0.        , 0.        , 0.        ]), array([0., 1., 0., 0., 0.]), array([1., 0., 0., 0., 0.])]


In [262]:
def load_and_process_test_data(input_directory, month, models=5, time_window=48):
    """
    Test 데이터를 로드하고, 모델 예측값과 True 값을 추출해 시간 단위로 이어붙이기기

    Args:
        input_directory (str): 모델별 test 데이터가 있는 디렉토리
        models (int): 모델 개수 (default=5)
        time_window (int): 샘플당 시간 창 (default=48)

    Returns:
        model_predictions (np.ndarray): (n_samples, models, time_window, 2)
        true_values (np.ndarray): (n_samples, time_window, 2)
    """
    model_predictions = []  # 모델별 예측값 저장
    true_values_list = []  # True U, True V 저장

    for model_idx in range(1, models + 1):
        model_dir = os.path.join(input_directory, f"model{model_idx}")
        
        model_data_list = []

        file_path = os.path.join(model_dir, f'test_month_{month}_model_{model_idx}_results.csv')
        df = pd.read_csv(file_path)

        # 모델 예측값 추출
        pred_u = df[f"Model {model_idx} Test Pred U"].values
        pred_v = df[f"Model {model_idx} Test Pred V"].values

        # 48시간 단위로 자르기
        n_samples = len(pred_u) // time_window
        pred_u = pred_u[:n_samples * time_window].reshape(n_samples, time_window, 1)
        pred_v = pred_v[:n_samples * time_window].reshape(n_samples, time_window, 1)

        model_data = np.concatenate([pred_u, pred_v], axis=2)  # (n_samples, 48, 2)
        model_data_list.append(model_data)

        # True U와 True V 추출 (첫 번째 모델 파일에서만 처리)
        if model_idx == 1:
            true_u = df["True U"].values[:n_samples * time_window].reshape(n_samples, time_window, 1)
            true_v = df["True V"].values[:n_samples * time_window].reshape(n_samples, time_window, 1)
            true_values_sample = np.concatenate([true_u, true_v], axis=2)
            true_values_list.append(true_values_sample)

        # 모델별 데이터 병합
        model_predictions.append(np.concatenate(model_data_list, axis=0))  # (total_samples, 48, 2)

    # 모델 예측값 형태 변환: (n_samples, models, 48, 2)
    model_predictions = np.stack(model_predictions, axis=1)

    # True Values 병합: (n_samples, 48, 2)
    true_values = np.concatenate(true_values_list, axis=0)

    return model_predictions, true_values


In [263]:
from sklearn.metrics import mean_squared_error
import numpy as np

def calculate_ensemble_rmse(model_predictions, true_values, weights):
    """
    최적 가중치를 사용하여 앙상블 RMSE를 계산

    Args:
        model_predictions (np.ndarray): 모델 예측값 (n_samples, models, time_window, 2)
        true_values (np.ndarray): 실제 값 (n_samples, time_window, 2)
        weights (np.ndarray): 최적 가중치 (models,)

    Returns:
        ensemble_rmse_dict (dict): 시간별 RMSE 딕셔너리
        overall_rmse (float): 전체 평균 RMSE
    """
    n_samples, n_models, time_window, _ = model_predictions.shape
    
    # 앙상블 예측값 계산 (가중치 적용)
    ensemble_predictions = np.tensordot(model_predictions, weights, axes=(1, 0))  # (n_samples, time_window, 2)

    # 시간별 RMSE를 저장할 딕셔너리
    ensemble_rmse_dict = {}
    
    for time_step in range(time_window):
        # 시간별 예측값과 실제값 추출
        preds = ensemble_predictions[:, time_step, :]
        trues = true_values[:, time_step, :]
        
        # RMSE 계산
        mse = mean_squared_error(trues, preds, multioutput='uniform_average')
        rmse = np.sqrt(mse)
        ensemble_rmse_dict[time_step] = round(rmse, 4)

    # 전체 평균 RMSE 계산
    overall_rmse = np.mean(list(ensemble_rmse_dict.values()))

    return ensemble_rmse_dict, round(overall_rmse, 4)

### Uniform RMSE

In [264]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

def calculate_uniform_ensemble_rmse(model_predictions, true_values):
    """
    동일 가중치 (0.2)로 앙상블 RMSE를 계산

    Args:
        model_predictions (np.ndarray): 모델 예측값 (n_samples, models, time_window, 2)
        true_values (np.ndarray): 실제 값 (n_samples, time_window, 2)

    Returns:
        ensemble_rmse_dict (dict): 시간별 RMSE 딕셔너리
        overall_rmse (float): 전체 평균 RMSE
    """
    n_samples, n_models, time_window, _ = model_predictions.shape
    
    # 동일 가중치 생성 (모든 모델에 대해 0.2로 설정)
    uniform_weights = np.full(n_models, 1 / n_models)  # [0.2, 0.2, 0.2, 0.2, 0.2]

    # 앙상블 예측값 계산 (가중치 적용)
    ensemble_predictions = np.tensordot(model_predictions, uniform_weights, axes=(1, 0))  # (n_samples, time_window, 2)

    # 시간별 RMSE를 저장할 딕셔너리
    ensemble_rmse_dict = {}
    for time_step in range(time_window):
        # 시간별 예측값과 실제값 추출
        preds = ensemble_predictions[:, time_step, :]
        trues = true_values[:, time_step, :]

        # RMSE 계산
        mse = mean_squared_error(trues, preds, multioutput='uniform_average')
        rmse = np.sqrt(mse)
        ensemble_rmse_dict[time_step] = round(rmse, 4)

    # 전체 평균 RMSE 계산
    overall_rmse = np.mean(list(ensemble_rmse_dict.values()))

    return ensemble_rmse_dict, round(overall_rmse, 4)

In [275]:
for i in optimal_weights_list:
    print(i)

[0.10123664 0.         0.74892202 0.14984134 0.        ]
[0.32101635 0.         0.         0.39395431 0.28502935]
[0.         0.         0.39715384 0.60284616 0.        ]
[0.11855262 0.88144738 0.         0.         0.        ]
[0.32416173 0.57812405 0.         0.09771422 0.        ]
[0.27629276 0.60595773 0.11774951 0.         0.        ]
[0.20503387 0.3296041  0.         0.         0.46536203]
[0.5640595  0.34967998 0.08626052 0.         0.        ]
[0.24724964 0.75275036 0.         0.         0.        ]
[0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0.]


### GA, Unifrom 평균

In [265]:
overall_rmse_list = []
uniform_overall_rmse_list = []

for month in range(1, 12):  # 1월부터 12월까지 반복
    try:
        # 테스트 데이터 로드 및 처리
        model_predictions, true_values = load_and_process_test_data(input_directory, month)

        # 월별 optimal_weights 확인
        if isinstance(optimal_weights_list, list):
            month_weights = optimal_weights_list[month - 1]
        else:
            raise ValueError("optimal_weights_list가 월별 가중치를 포함한 리스트가 아닙니다.")

        # 앙상블 RMSE 계산
        ensemble_rmse, overall_rmse = calculate_ensemble_rmse(model_predictions, true_values, month_weights)
        overall_rmse_list.append(overall_rmse)

        # Uniform 가중치 계산
        uniform_weights = np.full(len(model_predictions), 1 / len(model_predictions))  # 모델 개수에 따라 동적 생성
        uniform_ensemble_rmse, uniform_overall_rmse = calculate_uniform_ensemble_rmse(model_predictions, true_values)
        uniform_overall_rmse_list.append(uniform_overall_rmse)

        # 결과 출력
        print(f"\n{month}월 전체 앙상블 평균 RMSE: {overall_rmse}")
        print(f"\n{month}월 전체 Uniform 앙상블 평균 RMSE: {uniform_overall_rmse}")

        # 데이터프레임 생성 및 사용 (필요 시 확장 가능)
        ensemble_rmse_df = pd.DataFrame(list(ensemble_rmse.items()), columns=["Time Step", "RMSE"])
        uniform_rmse_df = pd.DataFrame(list(uniform_ensemble_rmse.items()), columns=["Time Step", "RMSE"])

    except Exception as e:
        print(f"{month}월 처리 중 오류 발생: {e}")

# 최종 평균 RMSE 출력
if overall_rmse_list:
    print("\n전체 월 평균 앙상블 RMSE: ", np.mean(overall_rmse_list))
else:
    print("\n앙상블 RMSE 결과가 없습니다.")

if uniform_overall_rmse_list:
    print("전체 월 평균 Uniform 앙상블 RMSE: ", np.mean(uniform_overall_rmse_list))
else:
    print("Uniform 앙상블 RMSE 결과가 없습니다.")



1월 전체 앙상블 평균 RMSE: 4.0966

1월 전체 Uniform 앙상블 평균 RMSE: 4.1519

2월 전체 앙상블 평균 RMSE: 4.5991

2월 전체 Uniform 앙상블 평균 RMSE: 4.4202

3월 전체 앙상블 평균 RMSE: 4.4697

3월 전체 Uniform 앙상블 평균 RMSE: 4.4632

4월 전체 앙상블 평균 RMSE: 4.0077

4월 전체 Uniform 앙상블 평균 RMSE: 3.8947

5월 전체 앙상블 평균 RMSE: 2.1396

5월 전체 Uniform 앙상블 평균 RMSE: 2.2408

6월 전체 앙상블 평균 RMSE: 3.081

6월 전체 Uniform 앙상블 평균 RMSE: 3.1044

7월 전체 앙상블 평균 RMSE: 1.9891

7월 전체 Uniform 앙상블 평균 RMSE: 2.0344

8월 전체 앙상블 평균 RMSE: 2.5418

8월 전체 Uniform 앙상블 평균 RMSE: 2.7378

9월 전체 앙상블 평균 RMSE: 2.2888

9월 전체 Uniform 앙상블 평균 RMSE: 2.3868

10월 전체 앙상블 평균 RMSE: 2.1686

10월 전체 Uniform 앙상블 평균 RMSE: 2.1511

11월 전체 앙상블 평균 RMSE: 3.2872

11월 전체 Uniform 앙상블 평균 RMSE: 3.1409

전체 월 평균 앙상블 RMSE:  3.151745454545455
전체 월 평균 Uniform 앙상블 RMSE:  3.156927272727273


### GA 검증 실험

In [266]:
def calculate_rmse(model_predictions, true_values, individual):
    """ 주어진 개체에 대해 앙상블 예측값을 계산하고 RMSE를 반환 """
    ensemble_prediction = np.sum([weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)], axis=0)
    rmse = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))
    return rmse

In [267]:
def genetic_algorithm_with_elitism(model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03, seed=None):
    """
    GA를 사용하여 최적의 가중치 찾기
    Args:
        model_predictions: (n_samples, models, time_window, 2) 형태의 모델 예측값
        true_values: (n_samples, time_window, 2) 형태의 실제값
        population_size: 초기 개체군 크기
        generations: 세대 수
        mutation_rate: 돌연변이 확률
        seed: 난수 시드를 고정하기 위한 값
    Returns:
        최적 가중치 배열
    """
    num_models = model_predictions.shape[1]  # 모델 수
    time_window = model_predictions.shape[2]  # 시간 단위
    
    # 초기 population 생성
    population = np.random.dirichlet(np.ones(num_models), size=population_size)  # 초기 가중치 개체군

    for generation in range(generations):
        fitness_scores = []

        # 현재 population의 fitness (RMSE) 계산
        for individual in population:
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            rmse = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))
            fitness_scores.append(rmse)
        
        fitness_scores = np.array(fitness_scores)

        # 새로운 population 생성
        new_population = []
        while len(new_population) < population_size:
            # 부모 선택
            np.random.seed(seed + generation)  # 세대마다 다른 시드 적용
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)

            # 교차 연산
            np.random.seed(seed + generation + 1)  # 세대마다 다른 시드 적용
            crossover_point = np.random.randint(1, num_models)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

            # 돌연변이
            if np.random.rand() < mutation_rate:
                np.random.seed(seed + generation + 2)  # 세대마다 다른 시드 적용
                mutation_vector = np.random.normal(0, 0.1, size=num_models)
                child = np.clip(child + mutation_vector, 0, 1)

            # 정규화
            child = child / np.sum(child)
            new_population.append(child)
        
        new_population = np.array(new_population)

        # 엘리티즘 적용: 상위 50% 개체 선택
        combined_population = np.vstack((population, new_population))
        combined_fitness = np.concatenate((fitness_scores, np.zeros(len(new_population))))

        for idx, individual in enumerate(new_population):
            ensemble_prediction = np.sum(
                [weight * model_predictions[:, i, :, :] for i, weight in enumerate(individual)],
                axis=0
            )
            combined_fitness[len(fitness_scores) + idx] = np.sqrt(np.mean((true_values - ensemble_prediction) ** 2))

        sorted_indices = np.argsort(combined_fitness)
        population = combined_population[sorted_indices][:population_size]

        print(f"Generation {generation + 1}/{generations}, Best RMSE: {combined_fitness[sorted_indices][0]}")

    # 최적 개체 반환
    best_weights = population[0]
    print(f"Optimal Weights: {best_weights}")
    return best_weights


In [268]:
import time
def run_ga_experiment(model_predictions, true_values, population_size=100, generations=100, mutation_rate=0.03, seed=None):
    """
    유전 알고리즘 실험을 한 번 실행하고, GA 계산 시간을 측정하여 최적 가중치를 반환합니다.
    """
    np.random.seed(seed)  # seed 설정
    
    start_time = time.time()

    optimal_weights = genetic_algorithm_with_elitism(model_predictions, true_values, 
                                                     population_size=population_size, 
                                                     generations=generations, mutation_rate=mutation_rate, seed=seed)

    ga_time = time.time() - start_time
    
    # 앙상블 예측
    ensemble_prediction = np.sum([weight * model_predictions[:, i, :, :] for i, weight in enumerate(optimal_weights)], axis=0)
    
    # 3D 배열을 2D 배열로 변환 (n_samples, time_window * 2)
    ensemble_prediction_2d = ensemble_prediction.reshape(-1, ensemble_prediction.shape[1] * ensemble_prediction.shape[2])
    true_values_2d = true_values.reshape(-1, true_values.shape[1] * true_values.shape[2])

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(true_values_2d, ensemble_prediction_2d))

    return optimal_weights, ga_time



In [269]:
from sklearn.metrics import mean_squared_error
import numpy as np

def calculate_ensemble_rmse2(model_predictions, true_values, weights):
    """
    최적 가중치를 사용하여 앙상블 RMSE를 계산

    Args:
        model_predictions (np.ndarray): 모델 예측값 (n_samples, models, time_window, 2)
        true_values (np.ndarray): 실제 값 (n_samples, time_window, 2)
        weights (np.ndarray): 최적 가중치 (models,)

    Returns:
        ensemble_rmse_dict (dict): 시간별 RMSE 딕셔너리
        overall_rmse (float): 전체 평균 RMSE
    """
    n_samples, n_models, time_window, _ = model_predictions.shape
    
    # 앙상블 예측값 계산 (가중치 적용)
    ensemble_predictions = np.tensordot(model_predictions, weights, axes=(1, 0))  # (n_samples, time_window, 2)

    # 시간별 RMSE를 저장할 딕셔너리
    ensemble_rmse_dict = {}
    
    for time_step in range(time_window):
        # 시간별 예측값과 실제값 추출
        preds = ensemble_predictions[:, time_step, :]
        trues = true_values[:, time_step, :]
        
        # RMSE 계산
        mse = mean_squared_error(trues, preds, multioutput='uniform_average')
        rmse = np.sqrt(mse)
        ensemble_rmse_dict[time_step] = round(rmse, 4)

    # 전체 평균 RMSE 계산
    overall_rmse = np.mean(list(ensemble_rmse_dict.values()))

    return round(overall_rmse, 4)

In [270]:
import pandas as pd
import numpy as np
import os

def save_to_csv_for_seed(seed, monthly_rmse_list, monthly_ga_time_list, output_dir="./ga_검증"):
    """각 seed별로 CSV 파일로 저장하는 함수"""
    
    valid_months = [i for i in range(1, 12)]
    
    # 월별 RMSE와 GA 계산 시간을 DataFrame으로 변환
    monthly_df_rmse = pd.DataFrame({
        'Seed': [seed],
        **{f'Month {month} RMSE': [monthly_rmse_list[idx]] for idx, month in enumerate(valid_months)}
    })
    
    monthly_df_ga_time = pd.DataFrame({
        'Seed': [seed],
        **{f'Month {month} GA Time': [monthly_ga_time_list[idx]] for idx, month in enumerate(valid_months)}
    })
    
    # 저장할 디렉토리 생성 (없는 경우 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 각 seed에 대해 결과 저장
    monthly_df_rmse.to_csv(f'{output_dir}/seed_{seed}_monthly_rmse.csv', index=False)
    monthly_df_ga_time.to_csv(f'{output_dir}/seed_{seed}_monthly_ga_time.csv', index=False)

    print(f"Seed {seed}의 CSV 파일들이 성공적으로 저장되었습니다.")



In [271]:
import numpy as np
import pandas as pd
import os

def calculate_seed_statistics(output_dir="./ga_검증"):
    """30개의 seed에 대한 전체 통계 (평균, 표준편차)를 계산하여 저장하는 함수"""
    all_rmse_list = []
    all_ga_time_list = []
    
    # 각 seed별 통계 계산을 위한 리스트
    seed_statistics = []

    for seed in range(31):  # 0부터 30까지 31번 반복 실험
        # seed별로 저장된 CSV 파일 불러오기
        monthly_df_rmse = pd.read_csv(f'{output_dir}/seed_{seed}_monthly_rmse.csv')
        monthly_df_ga_time = pd.read_csv(f'{output_dir}/seed_{seed}_monthly_ga_time.csv')
        
        # 전체 RMSE와 GA 시간의 값만 저장 (모든 월을 합친 통계)
        rmse_values = monthly_df_rmse.iloc[0, 1:].values  # 월별 RMSE
        ga_time_values = monthly_df_ga_time.iloc[0, 1:].values  # 월별 GA Time
        
        all_rmse_list.append(rmse_values)
        all_ga_time_list.append(ga_time_values)
        
        # 각 seed에 대한 전체 평균과 표준편차 계산
        seed_rmse_mean = np.mean(rmse_values)
        seed_ga_time_mean = np.mean(ga_time_values)
        
        # 각 seed별 통계를 저장
        seed_statistics.append({
            'Seed': seed,
            'Mean RMSE': seed_rmse_mean,
            'Mean GA Time': seed_ga_time_mean
        })
    
    # 전체 RMSE와 GA 시간에 대한 평균과 표준편차 계산
    total_rmse_mean = np.mean(all_rmse_list)
    total_ga_time_mean = np.mean(all_ga_time_list)

    # 최종 통계 DataFrame 생성 (전체 통계 + 각 seed별 통계)
    final_df = pd.DataFrame({
        'Total Mean RMSE': [total_rmse_mean],
        'Total Mean GA Time': [total_ga_time_mean],
    })

    # 각 seed별 통계를 DataFrame으로 변환
    seed_statistics_df = pd.DataFrame(seed_statistics)

    # 저장할 디렉토리 생성 (없는 경우 생성)
    os.makedirs(output_dir, exist_ok=True)

    # 전체 통계 결과를 CSV로 저장
    final_df.to_csv(f'{output_dir}/final_statistics.csv', index=False)
    # 각 seed별 통계 결과를 CSV로 저장
    seed_statistics_df.to_csv(f'{output_dir}/seed_statistics.csv', index=False)

    print(f"30개의 seed에 대한 통계 파일이 성공적으로 저장되었습니다.")


In [272]:
def validate_ga(input_directory, population_size=100, generations=100, mutation_rate=0.03, output_dir="./ga_검증"):
    """GA 가중치를 구하고, 테스트 데이터로 RMSE 계산"""
    overall_rmse_list = []
    ga_time_list = []

    for seed in range(31):  # 0부터 30까지 31번 반복 실험
        print(f"\nSeed {seed}로 실험 시작")
        
        np.random.seed(seed)  # 각 seed마다 난수 초기화
        
        monthly_rmse_list = []  # 매 `seed`마다 월별 RMSE 리스트 초기화
        monthly_ga_time_list = []  # 매 `seed`마다 월별 GA 계산 시간 리스트 초기화

        # 1월부터 12월까지 월별로 처리
        for month in range(1, 12):

            # 검증 데이터를 통해 GA 가중치 구하기
            model_predictions, true_values = load_and_process_validation_data(input_directory, month)  # 월별로 검증 데이터 로드
            optimal_weights, ga_time = run_ga_experiment(model_predictions, true_values, population_size, generations, mutation_rate, seed)
            
            # 테스트 데이터를 로드하여 최적 가중치로 예측 및 RMSE 계산
            test_predictions, test_true_values = load_and_process_test_data(input_directory, month)  # 월별로 테스트 데이터 로드
            test_rmse = calculate_ensemble_rmse2(test_predictions, test_true_values, optimal_weights)

            monthly_rmse_list.append(test_rmse)  # 월별 RMSE 추가
            monthly_ga_time_list.append(ga_time)  # GA 계산 시간도 리스트에 추가

        # 전체 실험 결과 (해당 seed에 대한 결과)
        overall_rmse_list.append(np.mean(monthly_rmse_list))  # 월별 RMSE 평균값을 overall에 추가
        ga_time_list.append(np.mean(monthly_ga_time_list))  # 월별 GA 계산 시간 평균값을 ga_time_list에 추가

        # 해당 seed에 대한 결과를 CSV로 저장
        save_to_csv_for_seed(seed, monthly_rmse_list.copy(), monthly_ga_time_list.copy(), output_dir)  # 복사된 리스트 전달

    # 모든 seed에 대해 통계 계산 후 CSV로 저장
    calculate_seed_statistics(output_dir)

    return overall_rmse_list, ga_time_list  # 결과 반환

In [273]:
# 실험에 사용할 입력 데이터 디렉토리 경로를 지정
input_directory = "./ga_w2"  # 데이터가 저장된 디렉토리 경로로 수정해주세요.
# validate_ga 함수 호출
# 실행 후 파일을 저장할 디렉토리 지정
output_dir = "./ga_검증_w2_2"  # 저장할 경로를 지정

# validate_ga 함수 실행하여 결과 가져오기
overall_rmse_list, ga_time_list, = validate_ga(input_directory, population_size=100, generations=100, mutation_rate=0.03, output_dir=output_dir)



Seed 0로 실험 시작
Generation 1/100, Best RMSE: 4.761268416323186
Generation 2/100, Best RMSE: 4.761268416323186
Generation 3/100, Best RMSE: 4.761268416323186
Generation 4/100, Best RMSE: 4.761268416323186
Generation 5/100, Best RMSE: 4.761268416323186
Generation 6/100, Best RMSE: 4.761268416323186
Generation 7/100, Best RMSE: 4.761268416323186
Generation 8/100, Best RMSE: 4.761268416323186
Generation 9/100, Best RMSE: 4.761268416323186
Generation 10/100, Best RMSE: 4.761268416323186
Generation 11/100, Best RMSE: 4.759915358486413
Generation 12/100, Best RMSE: 4.759915358486413
Generation 13/100, Best RMSE: 4.759915358486413
Generation 14/100, Best RMSE: 4.759915358486413
Generation 15/100, Best RMSE: 4.759915358486413
Generation 16/100, Best RMSE: 4.759915358486413
Generation 17/100, Best RMSE: 4.759915358486413
Generation 18/100, Best RMSE: 4.759915358486413
Generation 19/100, Best RMSE: 4.759915358486413
Generation 20/100, Best RMSE: 4.759915358486413
Generation 21/100, Best RMSE: 4.75

In [274]:
print("\n최종 평균 RMSE")
print(f"전체 앙상블 평균 RMSE: {np.mean(overall_rmse_list)}")
print(f"GA 평균 계산 시간: {np.mean(ga_time_list)}")


최종 평균 RMSE
전체 앙상블 평균 RMSE: 3.142510850439882
GA 평균 계산 시간: 1.362541737095002
